In [9]:
import torch
import transformers
import datasets
import sklearn
import matplotlib

print("All libraries installed successfully!")


All libraries installed successfully!


In [10]:
# 1. Install dependencies (run only if needed)
# !pip install torch transformers datasets scikit-learn matplotlib

# 2. Imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [11]:
# 3. Labels
emotion_labels = ["no emotion", "anger", "disgust", "fear", "happiness", "sadness", "surprise"]
intent_labels  = ["inform", "question", "directive", "commissive"]

In [ ]:
# 4. Load dataset (update paths as needed)

dataset = load_dataset(
    "csv",
    data_files={
        "train": r"C:\Users\NAMAN\Documents\GitHub\Prototype-\Friendly Dataset\train.csv",
        "test": r"C:\Users\NAMAN\Documents\GitHub\Prototype-\Friendly Dataset\test.csv",
        "validation": r"C:\Users\NAMAN\Documents\GitHub\Prototype-\Friendly Dataset\validation.csv"
    }
)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 11118
    })
    test: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
})


In [15]:
# 5. Tokenization
from transformers import BertTokenizer
from datasets import load_dataset
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


def encode_batch(batch):
    enc = tokenizer(batch["dialog"], truncation=True, padding="max_length", max_length=64)
    enc["emotion"] = batch["emotion"]  # already numeric
    enc["act"] = batch["act"]          # already numeric
    return enc


encoded = {}
for split in ["train", "test", "validation"]:
    encoded[split] = dataset[split].map(encode_batch, batched=True)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 4951.49 examples/s]


In [16]:
# 6. PyTorch Dataset
class ChatDataset(Dataset):
    def __init__(self, enc):
        self.enc = enc
    def __len__(self):
        return len(self.enc["input_ids"])
    def __getitem__(self, idx):
        item = {k: torch.tensor(self.enc[k][idx]) for k in ["input_ids", "attention_mask"]}
        item["emotion"] = torch.tensor(self.enc["emotion"][idx])
        item["act"] = torch.tensor(self.enc["act"][idx])
        return item

train_ds = ChatDataset(encoded["train"])
val_ds = ChatDataset(encoded["validation"])
test_ds = ChatDataset(encoded["test"])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)

In [19]:
import logging
from transformers.utils import logging as hf_logging

# Suppress HF info messages
hf_logging.set_verbosity_error()
logging.getLogger("hf_xet").setLevel(logging.CRITICAL)

In [22]:
import torch
from transformers import BertModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BERT on CPU first
bert = BertModel.from_pretrained("bert-base-uncased", cache_dir="C:/Users/NAMAN/.cache/huggingface")

# Initialize your model
model = ChatBERTModel(len(emotion_labels), len(intent_labels))
model.bert = bert

# Then move the model to GPU
model = model.to(device)


c:\Users\NAMAN\Documents\GitHub\Prototype-\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NAMAN\.cache\huggingface\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HT

KeyboardInterrupt: 

In [ ]:
import torch.nn as nn
from transformers import BertModel

class ChatBERTModel(nn.Module):
    def __init__(self, num_emotions, num_acts):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        hidden_size = self.bert.config.hidden_size
        self.emotion_classifier = nn.Linear(hidden_size, num_emotions)
        self.act_classifier = nn.Linear(hidden_size, num_acts)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output
        emo_logits = self.emotion_classifier(pooled)
        act_logits = self.act_classifier(pooled)
        return emo_logits, act_logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ChatBERTModel(len(emotion_labels), len(intent_labels)).to(device)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


KeyboardInterrupt: 

In [ ]:
# 8. Training/Eval functions
def train_epoch(loader):
    model.train()
    total_loss = 0
    for batch in loader:
        for k in batch:
            batch[k] = batch[k].to(device)
        emo_logits, act_logits = model(batch["input_ids"], batch["attention_mask"])
        loss_emo = nn.CrossEntropyLoss()(emo_logits, batch["emotion"])
        loss_act = nn.CrossEntropyLoss()(act_logits, batch["act"])
        loss = loss_emo + loss_act
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def eval_epoch(loader):
    model.eval()
    total_loss = 0
    emo_preds, emo_true, act_preds, act_true = [], [], [], []
    with torch.no_grad():
        for batch in loader:
            for k in batch:
                batch[k] = batch[k].to(device)
            emo_logits, act_logits = model(batch["input_ids"], batch["attention_mask"])
            loss_emo = nn.CrossEntropyLoss()(emo_logits, batch["emotion"])
            loss_act = nn.CrossEntropyLoss()(act_logits, batch["act"])
            loss = loss_emo + loss_act
            total_loss += loss.item()
            emo_preds.extend(emo_logits.argmax(1).cpu().numpy())
            emo_true.extend(batch["emotion"].cpu().numpy())
            act_preds.extend(act_logits.argmax(1).cpu().numpy())
            act_true.extend(batch["act"].cpu().numpy())
    return total_loss / len(loader), emo_preds, emo_true, act_preds, act_true

In [ ]:
# 9. Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
# 10. Training Loop
EPOCHS = 3
for epoch in range(EPOCHS):
    train_loss = train_epoch(train_loader)
    val_loss, emo_preds, emo_true, act_preds, act_true = eval_epoch(val_loader)
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

In [ ]:
# 11. Evaluation
print("Emotion classification report:")
print(classification_report(emo_true, emo_preds, target_names=emotion_labels))

print("Intent classification report:")
print(classification_report(act_true, act_preds, target_names=intent_labels))

cm = confusion_matrix(emo_true, emo_preds)
ConfusionMatrixDisplay(cm, display_labels=emotion_labels).plot(xticks_rotation=45)
plt.show()